In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/2023빅콘

/content/drive/MyDrive/Colab Notebooks/2023빅콘


In [3]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment',  None) # 복사 경고 무시

In [4]:
# 데이터 불러오기
data = pd.read_csv('2023빅콘테스트_어드밴스드리그_예술의전당.csv', encoding = 'UTF8')
df = pd.DataFrame(data)

In [5]:
df.columns

Index(['age', 'gender', 'membership_type_1', 'membership_type_2',
       'membership_type_3', 'membership_type_4', 'membership_type_5',
       'membership_type_6', 'tran_date', 'tran_time', 'play_date',
       'play_st_time', 'seat', 'price', 'ticket_cancel', 'discount_type',
       'performance_code', 'pre_open_date', 'open_date', 'genre', 'place',
       'running_time', 'intermission', 'member_yn'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920868 entries, 0 to 1920867
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   age                float64
 1   gender             object 
 2   membership_type_1  object 
 3   membership_type_2  object 
 4   membership_type_3  object 
 5   membership_type_4  object 
 6   membership_type_5  object 
 7   membership_type_6  float64
 8   tran_date          int64  
 9   tran_time          int64  
 10  play_date          int64  
 11  play_st_time       int64  
 12  seat               object 
 13  price              int64  
 14  ticket_cancel      int64  
 15  discount_type      object 
 16  performance_code   int64  
 17  pre_open_date      float64
 18  open_date          float64
 19  genre              object 
 20  place              object 
 21  running_time       int64  
 22  intermission       int64  
 23  member_yn          object 
dtypes: float64(4), int64(9), object(11)
memory usage: 

In [7]:
df.isna().sum()

age                  1300183
gender               1299714
membership_type_1    1299714
membership_type_2    1433215
membership_type_3    1749009
membership_type_4    1890675
membership_type_5    1919586
membership_type_6    1920868
tran_date                  0
tran_time                  0
play_date                  0
play_st_time               0
seat                       0
price                      0
ticket_cancel              0
discount_type              0
performance_code           0
pre_open_date         864385
open_date                340
genre                  25842
place                      0
running_time               0
intermission               0
member_yn                  0
dtype: int64

# 할인코드 범주 지정

In [8]:
dt = df['discount_type'].unique()
dt = pd.DataFrame(dt)
dt.columns = ['discount_type'] # 칼럼명 지정
dt['disc_t'] = '기타'         # 할인 종류 범주화 칼럼
dt['disc_p'] = '확인필요'     # 할인율 칼럼
dt['disc_p_int'] = 1.0        # 할인율 수치화

In [9]:
dt.head()

,discount_type,disc_t,disc_p,disc_p_int
0,일반,기타,확인필요,1.0
1,초대권,기타,확인필요,1.0
2,블루회원 할인20%,기타,확인필요,1.0
3,골드회원 할인20%,기타,확인필요,1.0
4,기획사,기타,확인필요,1.0


## 할인율 지정

In [10]:
dis_p_index = dt[dt['discount_type'].str.contains('%')].index
dt.iloc[dis_p_index,2] = dt.iloc[dis_p_index,0].str[-3:]

In [11]:
dt['disc_p'].unique() # 할인율 정리

array(['확인필요', '20%', '30%', '40%', '10%', '50%', '인5%', '15%', '25%',
       '60%', ')5%', '35%', '12%', '0%_', '%할인', '75%', '80%'],
      dtype=object)

In [12]:
# 할인율이 5%인 내용 정리

five_p = ['인5%', '\)5%'] # \)인 이유 : 괄호가 중복되어 오류 발생

for s in five_p :
  dis_p_index_5 = dt[dt['discount_type'].str.contains(s)].index
  dt.iloc[dis_p_index_5,2] = '5%'

In [13]:
# '0%_' 정리

dis_p_index_ = dt[dt['disc_p'] == '0%_'].index
dt.iloc[dis_p_index_,2] = dt.iloc[dis_p_index_,0].str[-4:-1]

In [14]:
# '%할인' 정리

dis_p_index_dis = dt[dt['disc_p'] == '%할인'].index
dt.iloc[dis_p_index_dis,2] = dt.iloc[dis_p_index_dis,0].str[-5:-2]

In [15]:
dt.head(10) # disc_p (할인율 정리)

,discount_type,disc_t,disc_p,disc_p_int
0,일반,기타,확인필요,1.0
1,초대권,기타,확인필요,1.0
2,블루회원 할인20%,기타,20%,1.0
3,골드회원 할인20%,기타,20%,1.0
4,기획사,기타,확인필요,1.0
5,K-lang 멤버십 회원(1인2매)20%,기타,20%,1.0
6,기획사판매,기타,확인필요,1.0
7,골드회원 할인30%,기타,30%,1.0
8,R석 기획사 할인,기타,확인필요,1.0
9,싹틔우미 할인40%,기타,40%,1.0


## 할인율 % 정리

In [16]:
dis_p_int_index = dt[dt['disc_p'].str.contains('%')].index
dt.iloc[dis_p_int_index,3] = dt.iloc[dis_p_int_index,2].str[:-1]

In [17]:
dt['disc_p_int'].unique() # disc_p_int (할인율 % 정리)

array([1.0, '20', '30', '40', '10', '50', '5', '15', '25', '60', '35',
       '12', '75', '80'], dtype=object)

In [18]:
dt['disc_p_int'][dt['disc_p_int'] != 1] = pd.to_numeric(dt['disc_p_int']) / 100.0

In [19]:
dt.sample(10)

,discount_type,disc_t,disc_p,disc_p_int
214,조기예매할인(1인4매/5월28일까지)20%,기타,20%,0.2
579,H-Point 회원 할인(어플 현장 인증)20%,기타,20%,0.2
548,스텝할인20%,기타,20%,0.2
106,후원회원 할인30%,기타,30%,0.3
306,임산부(동반1인)50%,기타,50%,0.5
805,가정의달 기념 가족할인(3인이상구매시)30%,기타,30%,0.3
402,싹딜가(골드)40%,기타,40%,0.4
65,문화햇살,기타,확인필요,1.0
470,조기예매할인(1인10매/7월3일까지)20%,기타,20%,0.2
855,그린카드(1인2매)20%,기타,20%,0.2


## 할인 종류 정리

In [20]:
# 할인 종류 딕셔너리
disc_dict = {'경로' : '경로우대',
             '실버' : '경로우대',
             '65세' : '경로우대',
             '기획사' : '기획사',
             '초대' : '초대',
             '노블회원' : '멤버쉽',
             '블루회원' : '멤버쉽',
             '싹틔우미' : '멤버쉽',
             '골드회원' : '멤버쉽',
             '그린회원' : '멤버쉽',
             '코로나' : '코로나',
             '후원' : '후원',
             '장애인' : '장애인 및 국가유공자',
             '유공자' : '장애인 및 국가유공자',
             '초,중,고,' : '학생',
             '초/중/고/' : '학생',
             '대학' : '학생',
             '청소년' : '학생',
             '릴레이' : '릴레이', # 문화릴레이 : 이전에 공연을 관림 시 다음 공연 할인
             '조기' : '조기예매',
             '가족' : '가족',
             '패밀리' : '가족',
             '얼리버드' : '조기예매',
             '카드' : '카드'}

In [21]:
for keys, values in disc_dict.items() :
  dis_t_index = dt[dt['discount_type'].str.contains(keys)].index
  dt.iloc[dis_t_index,1] = values

In [22]:
dt['disc_t'].value_counts() # 13개의 범주

기타             378
조기예매           142
학생             124
경로우대            61
카드              49
멤버쉽             41
장애인 및 국가유공자     41
기획사             21
릴레이             21
가족              15
후원              12
코로나             11
초대               6
Name: disc_t, dtype: int64

In [23]:
dt.head(10)

,discount_type,disc_t,disc_p,disc_p_int
0,일반,기타,확인필요,1.0
1,초대권,초대,확인필요,1.0
2,블루회원 할인20%,멤버쉽,20%,0.2
3,골드회원 할인20%,멤버쉽,20%,0.2
4,기획사,기획사,확인필요,1.0
5,K-lang 멤버십 회원(1인2매)20%,기타,20%,0.2
6,기획사판매,기획사,확인필요,1.0
7,골드회원 할인30%,멤버쉽,30%,0.3
8,R석 기획사 할인,기획사,확인필요,1.0
9,싹틔우미 할인40%,멤버쉽,40%,0.4


In [24]:
df = pd.merge(df, dt)
df.columns

Index(['age', 'gender', 'membership_type_1', 'membership_type_2',
       'membership_type_3', 'membership_type_4', 'membership_type_5',
       'membership_type_6', 'tran_date', 'tran_time', 'play_date',
       'play_st_time', 'seat', 'price', 'ticket_cancel', 'discount_type',
       'performance_code', 'pre_open_date', 'open_date', 'genre', 'place',
       'running_time', 'intermission', 'member_yn', 'disc_t', 'disc_p',
       'disc_p_int'],
      dtype='object')

# 멤버십 수치화

In [25]:
mt = set(np.concatenate([df['membership_type_1'].unique(), df['membership_type_2'].unique(),df['membership_type_3'].unique(),df['membership_type_4'].unique(),df['membership_type_5'].unique(),df['membership_type_6'].unique(),]))

In [26]:
mt # membership_type_1 ~ 6의 원자값

{nan, nan, '골드', '그린', '노블', '무료', '블루', '싹틔우미'}

In [27]:
membership = ['membership_type_1', 'membership_type_2', 'membership_type_3', 'membership_type_4', 'membership_type_5', 'membership_type_6']

In [28]:
missing_fill_val = {membership[0] : 0,
                    membership[1] : 0,
                    membership[2] : 0,
                    membership[3] : 0,
                    membership[4] : 0,
                    membership[5] : 0 }

df = df.fillna(missing_fill_val)

df.head()

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,disc_t,disc_p,disc_p_int
0,50.0,F,골드,무료,그린,0,0,0.0,20220114,1512,...,20220114.0,20220115.0,교향곡,콘서트홀,120,15,Y,기타,확인필요,1.0
1,50.0,M,무료,0,0,0,0,0.0,20220206,1615,...,20220109.0,20220110.0,독주,콘서트홀,90,0,Y,기타,확인필요,1.0
2,NaN,F,블루,무료,0,0,0,0.0,20190703,908,...,NaN,20190413.0,클래식,콘서트홀,100,15,N,기타,확인필요,1.0
3,NaN,NaN,0,0,0,0,0,0.0,20220624,1618,...,20220430.0,20220430.0,교향곡,콘서트홀,110,15,N,기타,확인필요,1.0
4,50.0,M,골드,무료,0,0,0,0.0,20221030,1645,...,20221105.0,20221030.0,교향곡,콘서트홀,120,20,Y,기타,확인필요,1.0


In [29]:
mt = set(np.concatenate([df['membership_type_1'].unique(), df['membership_type_2'].unique(),df['membership_type_3'].unique(),df['membership_type_4'].unique(),df['membership_type_5'].unique(),df['membership_type_6'].unique(),]))
mt

{0, '골드', '그린', '노블', '무료', '블루', '싹틔우미'}

In [30]:
membership_dict = {'무료' : 1,
                   '노블' : 1,
                   '싹틔우미' : 1,
                   '그린' : 2, # 연회비 2만원
                   '블루' : 3, # 연회비 4만원
                   '골드' : 4} # 연회비 10만원

In [31]:
for keys, values in membership_dict.items() :
  df[membership] = df[membership].replace(keys, values)

In [32]:
mt = set(np.concatenate([df['membership_type_1'].unique(), df['membership_type_2'].unique(),df['membership_type_3'].unique(),df['membership_type_4'].unique(),df['membership_type_5'].unique(),df['membership_type_6'].unique(),]))
mt

{0.0, 1.0, 2.0, 3.0, 4.0}

## 가장 높은 멤버십만 사용

In [33]:
df['membership_type'] = df[membership].max(axis = 1)

In [34]:
df['membership_type'].value_counts()

0.0    1299714
4.0     238199
1.0     210547
2.0      95230
3.0      77178
Name: membership_type, dtype: int64

In [35]:
df

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,open_date,genre,place,running_time,intermission,member_yn,disc_t,disc_p,disc_p_int,membership_type
0,50.0,F,4,1,2,0,0,0.0,20220114,1512,...,20220115.0,교향곡,콘서트홀,120,15,Y,기타,확인필요,1.0,4.0
1,50.0,M,1,0,0,0,0,0.0,20220206,1615,...,20220110.0,독주,콘서트홀,90,0,Y,기타,확인필요,1.0,1.0
2,NaN,F,3,1,0,0,0,0.0,20190703,908,...,20190413.0,클래식,콘서트홀,100,15,N,기타,확인필요,1.0,3.0
3,NaN,NaN,0,0,0,0,0,0.0,20220624,1618,...,20220430.0,교향곡,콘서트홀,110,15,N,기타,확인필요,1.0,0.0
4,50.0,M,4,1,0,0,0,0.0,20221030,1645,...,20221030.0,교향곡,콘서트홀,120,20,Y,기타,확인필요,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920863,50.0,F,1,0,0,0,0,0.0,20200816,953,...,20200704.0,교향곡,콘서트홀,100,15,Y,경로우대,50%,0.5,1.0
1920864,70.0,F,3,4,1,1,0,0.0,20200703,1119,...,20200704.0,교향곡,콘서트홀,100,15,Y,경로우대,50%,0.5,4.0
1920865,70.0,F,3,4,1,1,0,0.0,20200703,1119,...,20200704.0,교향곡,콘서트홀,100,15,Y,경로우대,50%,0.5,4.0
1920866,70.0,F,4,1,1,2,0,0.0,20200704,855,...,20200704.0,교향곡,콘서트홀,100,15,Y,경로우대,50%,0.5,4.0


# play_date -> 연도 + 분기로 범주

In [36]:
df['play_date'] = df['play_date'].astype('str')

df['play_date_year'] = df['play_date'].str.slice(start=0, stop=4)
df['play_date_month'] = df['play_date'].str.slice(start=4, stop=6).astype(int)

In [37]:
print(df[['play_date_year', 'play_date_month']])

        play_date_year  play_date_month
0                 2022                2
1                 2022                3
2                 2019                7
3                 2022                6
4                 2022               12
...                ...              ...
1920863           2020                8
1920864           2020                8
1920865           2020                8
1920866           2020                8
1920867           2020                8

[1920868 rows x 2 columns]


In [38]:
def map_month_to_quarter(month):
    month = int(month)  # 'play_date_month' 열을 정수로 변환
    if 1 <= month <= 3:
        return '_1'
    elif 4 <= month <= 6:
        return '_2'
    elif 7 <= month <= 9:
        return '_3'
    else:
        return '_4'

df['play_date_quarter'] = df['play_date_month'].map(map_month_to_quarter)
df['play_date_quarter'] = df['play_date_year'] + df['play_date_quarter']

# Seats 층 ,블록 추출

## 층

In [39]:
df['place'].unique()

array(['콘서트홀', 'IBK챔버홀', '리사이틀홀'], dtype=object)

In [40]:
df['seat']

0            3층 BOX9 10
1           1층 B블록12열 7
2           1층 C블록17열 3
3            3층 BOX12 3
4            2층 D블록8열 3
               ...     
1920863    합창석 G블록3열 32
1920864     1층 C블록9열 10
1920865      1층 C블록9열 7
1920866      3층 A블록1열 4
1920867     합창석 H블록4열 8
Name: seat, Length: 1920868, dtype: object

In [41]:
# 좌석 층
df['seat_f'] = df['seat'].str.split(" ").str[0]

In [42]:
df['seat_f']

0           3층
1           1층
2           1층
3           3층
4           2층
          ... 
1920863    합창석
1920864     1층
1920865     1층
1920866     3층
1920867    합창석
Name: seat_f, Length: 1920868, dtype: object

## 블록 (BOX, 블록)

In [43]:
# 좌석 열

df['seat_c'] = ""

In [44]:
# 좌석 블록

df['seat_b'] = df['seat'].str.split(" ").str[1]

In [45]:
df['seat_b']

0            BOX9
1          B블록12열
2          C블록17열
3           BOX12
4           D블록8열
            ...  
1920863     G블록3열
1920864     C블록9열
1920865     C블록9열
1920866     A블록1열
1920867     H블록4열
Name: seat_b, Length: 1920868, dtype: object

In [46]:
df[df['place'] == '리사이틀홀']['seat'].str.split(" ").str[1].unique()

array(['3열', '1열', '4열', '9열', '2열', '7열', '10열', 'BOX2', '5열', '8열',
       '6열', '11열', 'BOX1', '12열', 'BOX4', 'BOX3', '우측', '좌측'],
      dtype=object)

In [47]:
# 리사이틀홀 열 추출

seat_C = df[(df['place'] == '리사이틀홀') & (df['seat'].str.contains('열'))].index
df.loc[seat_C, 'seat_c'] = df['seat_b'].str[0:-1]

seat_C = df[(df['place'] == '리사이틀홀') & (df['seat'].str.contains('BOX'))].index
df.loc[seat_C, 'seat_c'] = df['seat_b'].str[-1]

seat_C = df[(df['place'] == '리사이틀홀') & (df['seat'].str.contains('우측|좌측'))].index
df.loc[seat_C, 'seat_c'] = df['seat'].str.split(" ").str[2].str[0]

In [48]:
seat_b_box = df[(df['place'] == '리사이틀홀') & (df['seat'].str.contains('열'))].index
df.loc[seat_b_box, 'seat_b'] = "BLOCK"

seat_b_box = df[(df['place'] == '리사이틀홀') & (df['seat'].str.contains('BOX|우측|좌측'))].index
df.loc[seat_b_box, 'seat_b'] = 'BOX'

In [49]:
df[df['place'] == '리사이틀홀'][['seat', 'seat_b', 'seat_c']].sample(10)

,seat,seat_b,seat_c
716875,1층 7열 6,BLOCK,7
873246,2층 BOX4 1열 15,BOX,4
826880,1층 BOX2 1열 11,BOX,2
313539,2층 1열 17,BLOCK,1
39624,1층 5열 17,BLOCK,5
467799,1층 11열 3,BLOCK,11
1372014,1층 8열 14,BLOCK,8
732728,1층 2열 5,BLOCK,2
1757334,1층 5열 12,BLOCK,5
1108150,1층 BOX1 1열 10,BOX,1


In [50]:
# 콘서트홀 블록 추출

seat_b_box = df[(df['place'] == '콘서트홀') & (df['seat'].str.contains('BOX|우측|좌측'))].index
df.loc[seat_b_box, 'seat_b'] = 'BOX'

seat_b_box = df[(df['place'] == '콘서트홀') & (df['seat'].str.contains('열'))].index
df.loc[seat_b_box, 'seat_b'] = df.loc[seat_b_box, 'seat_b'].str[0]

In [51]:
# 콘서트홀 열 추출

seat_C = df[(df['place'] == '콘서트홀') & (df['seat'].str.contains('열'))].index
df.loc[seat_C, 'seat_c'] = df['seat'].str.split(" ").str[1].str[3:-1]

seat_C = df[(df['place'] == '콘서트홀') & (df['seat'].str.contains('BOX'))].index
df.loc[seat_C, 'seat_c'] = df['seat'].str.split(" ").str[1].str[-1]

In [52]:
df[df['place'] == '콘서트홀'][['seat', 'seat_b', 'seat_c']].sample(10)

,seat,seat_b,seat_c
1512414,2층 E블록6열 5,E,6
762396,2층 C블록7열 5,C,7
306822,1층 D블록15열 8,D,15
1315126,1층 D블록22열 3,D,22
1075738,1층 D블록13열 9,D,13
362553,1층 A블록11열 8,A,11
799019,1층 E블록18열 5,E,18
1738489,3층 E블록7열 3,E,7
1436395,3층 M블록2열 1,M,2
57925,3층 C블록4열 6,C,4


In [53]:
# IBK챔버홀 블록 추출
seat_b_box = df[(df['place'] == 'IBK챔버홀') & (df['seat'].str.contains('BOX|우측|좌측'))].index
df.loc[seat_b_box, 'seat_b'] = 'BOX'

seat_b_box = df[(df['place'] == 'IBK챔버홀') & (df['seat'].str.contains('열'))].index
df.loc[seat_b_box, 'seat_b'] = df.loc[seat_b_box, 'seat_b'].str[0]

In [54]:
# 콘서트홀 열 추출

seat_C = df[(df['place'] == 'IBK챔버홀') & (df['seat'].str.contains('열'))].index
df.loc[seat_C, 'seat_c'] = df['seat'].str.split(" ").str[2].str[0:-1]

seat_C = df[(df['place'] == 'IBK챔버홀') & (df['seat'].str.contains('BOX'))].index
df.loc[seat_C, 'seat_c'] = df['seat'].str.split(" ").str[1].str[-1]

In [55]:
df[df['place'] == 'IBK챔버홀'][['seat', 'seat_b', 'seat_c']].sample(10)

,seat,seat_b,seat_c
91493,2층 B블록 4열 9,B,4
817998,1층 B블록 1열 9,B,1
1500788,2층 B블록 1열 3,B,1
1012861,2층 B블록 4열 7,B,4
1168449,1층 B블록 2열 4,B,2
1430883,1층 B블록 12열 10,B,12
1212278,1층 B블록 10열 10,B,10
1009898,2층 B블록 2열 10,B,2
1732079,1층 B블록 13열 3,B,13
690639,2층 A블록 5열 1,A,5


In [56]:
df[['place','seat','seat_f','seat_b','seat_c']].sample(10)

,place,seat,seat_f,seat_b,seat_c
560326,IBK챔버홀,1층 A블록 13열 1,1층,A,13
720857,IBK챔버홀,2층 A블록 4열 7,2층,A,4
1436222,콘서트홀,1층 B블록2열 6,1층,B,2
853879,리사이틀홀,1층 우측 1열 6,1층,BOX,1
556933,콘서트홀,2층 E블록6열 12,2층,E,6
1850785,IBK챔버홀,2층 A블록 2열 13,2층,A,2
858930,리사이틀홀,1층 BOX1 2열 22,1층,BOX,1
322057,콘서트홀,1층 C블록14열 9,1층,C,14
1277234,리사이틀홀,1층 9열 5,1층,BLOCK,9
1391912,리사이틀홀,2층 3열 10,2층,BLOCK,3


In [57]:
df.loc[df['seat_f'] == '1층', 'seat_f'] = 1
df.loc[df['seat_f'] == '2층', 'seat_f'] = 2
df.loc[df['seat_f'] == '3층', 'seat_f'] = 3
df.loc[df['seat_f'] == '합창석', 'seat_f'] = 0 # 합창단 = 0

In [58]:
df['seat_f'].astype(int)

0          3
1          1
2          1
3          3
4          2
          ..
1920863    0
1920864    1
1920865    1
1920866    3
1920867    0
Name: seat_f, Length: 1920868, dtype: int64

In [59]:
df[['place','seat','seat_f','seat_b','seat_c']].sample(10)

,place,seat,seat_f,seat_b,seat_c
82939,콘서트홀,3층 E블록5열 1,3,E,5
934212,IBK챔버홀,1층 C블록 15열 3,1,C,15
276529,콘서트홀,1층 C블록11열 2,1,C,11
223685,IBK챔버홀,1층 B블록 2열 8,1,B,2
802138,IBK챔버홀,1층 C블록 16열 5,1,C,16
155025,IBK챔버홀,1층 B블록 16열 10,1,B,16
1010212,IBK챔버홀,1층 B블록 17열 11,1,B,17
1017616,IBK챔버홀,1층 C블록 9열 6,1,C,9
1109834,리사이틀홀,2층 3열 8,2,BLOCK,3
1571225,콘서트홀,1층 A블록9열 11,1,A,9


# 범주형 변수 라벨링

# 필요없는 데이터 삭제

In [60]:
module_6 = []
performance_code_list = sorted(df['performance_code'].unique().tolist())
code_list = []
for i in performance_code_list:
  if df[df['performance_code'] == i]['ticket_cancel'].nunique() == 1:
    code_list.append(i)
code_list_result = []
for j in code_list:
  if df[df['performance_code'] == j]['ticket_cancel'].sum() != 0:
    code_list_result.append(j)

for i in code_list_result:
  module_6.append(df[df['performance_code'] == i].index.tolist())

new_list = [] # 빈 리스트 생성
for i in range(len(module_6)):
  for v in module_6[i]:
    if v not in new_list:
        new_list.append(v)
module_6 = new_list
module_6 = sorted(module_6) # module_6은 전체 취소된 공연의 공연코드에 해당하는 인덱스

In [61]:
module_7 = df[df['genre'].isna()].index # 장르가 없는 데이터

In [62]:
module_8 = df[df['open_date'].isna()].index # open_date가 없는 데이터

In [63]:
result = []
result.append(module_6)
result.append(module_7)
result.append(module_8)

new_list = [] # 빈 리스트 생성
for i in range(len(result)):
  for v in result[i]:
    if v not in new_list:
        new_list.append(v)
result = new_list
result = sorted(result)

len(result)

26430

In [64]:
df.drop(index = result, axis = 0, inplace = True)
df.reset_index(drop = True, inplace = True)

df

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,...,disc_t,disc_p,disc_p_int,membership_type,play_date_year,play_date_month,play_date_quarter,seat_f,seat_c,seat_b
0,50.0,F,4,1,2,0,0,0.0,20220114,1512,...,기타,확인필요,1.0,4.0,2022,2,2022_1,3,9,BOX
1,50.0,M,1,0,0,0,0,0.0,20220206,1615,...,기타,확인필요,1.0,1.0,2022,3,2022_1,1,12,B
2,NaN,F,3,1,0,0,0,0.0,20190703,908,...,기타,확인필요,1.0,3.0,2019,7,2019_3,1,17,C
3,NaN,NaN,0,0,0,0,0,0.0,20220624,1618,...,기타,확인필요,1.0,0.0,2022,6,2022_2,3,2,BOX
4,50.0,M,4,1,0,0,0,0.0,20221030,1645,...,기타,확인필요,1.0,4.0,2022,12,2022_4,2,8,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894433,50.0,F,1,0,0,0,0,0.0,20200816,953,...,경로우대,50%,0.5,1.0,2020,8,2020_3,0,3,G
1894434,70.0,F,3,4,1,1,0,0.0,20200703,1119,...,경로우대,50%,0.5,4.0,2020,8,2020_3,1,9,C
1894435,70.0,F,3,4,1,1,0,0.0,20200703,1119,...,경로우대,50%,0.5,4.0,2020,8,2020_3,1,9,C
1894436,70.0,F,4,1,1,2,0,0.0,20200704,855,...,경로우대,50%,0.5,4.0,2020,8,2020_3,3,1,A


In [65]:
len(df.columns)

34

# 필요없는 칼럼 제거

In [66]:
df.isna().sum()

age                  1285013
gender               1284573
membership_type_1          0
membership_type_2          0
membership_type_3          0
membership_type_4          0
membership_type_5          0
membership_type_6          0
tran_date                  0
tran_time                  0
play_date                  0
play_st_time               0
seat                       0
price                      0
ticket_cancel              0
discount_type              0
performance_code           0
pre_open_date         863031
open_date                  0
genre                      0
place                      0
running_time               0
intermission               0
member_yn                  0
disc_t                     0
disc_p                     0
disc_p_int                 0
membership_type            0
play_date_year             0
play_date_month            0
play_date_quarter          0
seat_f                     0
seat_c                     0
seat_b                     0
dtype: int64

In [67]:
df.columns

Index(['age', 'gender', 'membership_type_1', 'membership_type_2',
       'membership_type_3', 'membership_type_4', 'membership_type_5',
       'membership_type_6', 'tran_date', 'tran_time', 'play_date',
       'play_st_time', 'seat', 'price', 'ticket_cancel', 'discount_type',
       'performance_code', 'pre_open_date', 'open_date', 'genre', 'place',
       'running_time', 'intermission', 'member_yn', 'disc_t', 'disc_p',
       'disc_p_int', 'membership_type', 'play_date_year', 'play_date_month',
       'play_date_quarter', 'seat_f', 'seat_c', 'seat_b'],
      dtype='object')

In [68]:
len(df.columns)

34

In [69]:
df.drop(columns=['age', 'gender', 'membership_type_1','membership_type_2',
                    'membership_type_3', 'membership_type_4', 'membership_type_5',
                    'membership_type_6', 'discount_type', 'play_date', 'play_date_year', 'play_date_month',
                    'pre_open_date', 'tran_date', 'performance_code','seat','disc_p'], inplace=True)

In [70]:
len(df.columns)

17

In [71]:
df['place'].unique()

array(['콘서트홀', 'IBK챔버홀', '리사이틀홀'], dtype=object)

In [72]:
df_concet = df[df['place'] == '콘서트홀']
df_recital = df[df['place'] == '리사이틀홀']
df_ibk = df[df['place'] == 'IBK챔버홀']

In [141]:
# df_concet.to_csv('Data/concet.csv', index = False, encoding="utf-8-sig")

In [142]:
# df_recital.to_csv('Data/recital.csv', index = False, encoding="utf-8-sig")

In [143]:
# df_ibk.to_csv('Data/ibk.csv', index = False, encoding="utf-8-sig")